In [1]:
from tweepy import Stream
import pandas as pd
import numpy as np

In [2]:
# Import the Twython class
from twython import Twython  
import json

# Load credentials from json file
with open("twitter_credentials.json", "r") as file:  
     credentials = json.load(file)

# Instantiate an object
python_tweets = Twython( credentials['CONSUMER_KEY'],  credentials['CONSUMER_SECRET'])





In [5]:
import tweepy
auth = tweepy.OAuthHandler(credentials['CONSUMER_KEY'], credentials['CONSUMER_SECRET'])
auth.set_access_token(credentials['ACCESS_TOKEN'], credentials['ACCESS_SECRET'])

api = tweepy.API(auth)

In [6]:

from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import time
import argparse
import string
import json


class MyListener(StreamListener):
    """Custom StreamListener for streaming data."""

    def __init__(self, data_dir, query):
        query_fname = format_filename(query)
        self.outfile = "%s/stream_%s.json" % (data_dir, query_fname)

    def on_data(self, data):
        try:
            with open(self.outfile, 'a') as f:
                f.write(data)
                #print(data)
                return True
        except BaseException as e:
            #print("Error on_data: %s" % str(e))
            time.sleep(5)
        return True

    def on_error(self, status):
        print(status)
        return True


In [3]:

def format_filename(fname):
    """Convert file name into a safe string.
    Arguments:
        fname -- the file name to convert
    Return:
        String -- converted file name
    """
    return ''.join(convert_valid(one_char) for one_char in fname)


def convert_valid(one_char):
    """Convert a character into '_' if invalid.
    Arguments:
        one_char -- the char to convert
    Return:
        Character -- converted char
    """
    valid_chars = "-_.%s%s" % (string.ascii_letters, string.digits)
    if one_char in valid_chars:
        return one_char
    else:
        return '_'

In [9]:
query = 'politics'
data_dir = '/Users/shahla/Dropbox/SharpestMinds/stream/data'
#MyListener() saves the data into a .json file with name stream_query
twitter_stream = Stream(auth, MyListener(data_dir, query),tweet_mode='extended')
twitter_stream.filter(track=[query])

KeyboardInterrupt: 

In [6]:

df = pd.read_json("/Users/shahla/Dropbox/SharpestMinds/stream/data/stream_politics.json",lines=True)


KeyboardInterrupt: 

In [ ]:
df.shape

In [ ]:
list_columns=list(df.columns)
list_columns

In [ ]:
df.created_at[1:10]

In [151]:
#keep English languages
data=df[df.lang=='en']


In [152]:
data.shape

(69743, 37)

In [16]:
data.truncated[21]

False

In [17]:

data.text[21]

'RT @gaywonk: This will reward bad actors -- conspiracy theorists and neo-nazis -- who will game the system to get their tweets included in…'

In [18]:
data.retweeted_status[32270]

nan

In [19]:
data.retweeted_status[21]

{'created_at': 'Tue Apr 16 20:36:00 +0000 2019',
 'id': 1118251717336997888,
 'id_str': '1118251717336997888',
 'text': 'This will reward bad actors -- conspiracy theorists and neo-nazis -- who will game the system to get their tweets i… https://t.co/AnZq8weznX',
 'display_text_range': [0, 140],
 'source': '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
 'truncated': True,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 323599188,
  'id_str': '323599188',
  'name': 'Carlos Maza',
  'screen_name': 'gaywonk',
  'location': 'New York, NY',
  'url': 'http://www.carlosmmaza.com/',
  'description': 'I write Strikethrough, a video series about the media, for @voxdotcom. Tucker Carlson is a white supremacist. He/him. IG: gaywonk',
  'translator_type': 'none',
  'protected': False,
  'verified': True,
  'followers_count': 75741,
  'friends_c

In [29]:
data.retweeted_status[21]['extended_tweet']['full_text']

'This will reward bad actors -- conspiracy theorists and neo-nazis -- who will game the system to get their tweets included in general topics like "politics" and "immigration" and reach millions of new users: https://t.co/vfojIXU0wU'

In [88]:
data.text[31]

'Notley needs to resign, get out of politics. She failed Alberta, and she failed her party as leader... she was no l… https://t.co/wCBIx500SE'

In [22]:
data.retweeted_status[31]

nan

In [89]:
data.extended_tweet[31]

{'full_text': 'Notley needs to resign, get out of politics. She failed Alberta, and she failed her party as leader... she was no leader.\n\n#ableg',
 'display_text_range': [0, 129],
 'entities': {'hashtags': [{'text': 'ableg', 'indices': [123, 129]}],
  'urls': [],
  'user_mentions': [],
  'symbols': []}}

In [90]:
data.extended_tweet[31]['full_text']

'Notley needs to resign, get out of politics. She failed Alberta, and she failed her party as leader... she was no leader.\n\n#ableg'

In [91]:
data.created_at[0:3]

0   2019-04-17 14:39:20
1   2019-04-17 14:39:20
2   2019-04-17 14:39:21
Name: created_at, dtype: datetime64[ns]

In [142]:
tweets=pd.DataFrame(columns=['time','tweet'],index=data.index)


NameError: name 'data' is not defined

In [24]:
for i in data.index:   
       if pd.isnull(data.retweeted_status[i]):
             if pd.isnull(data.extended_tweet[i]):
                    tweets.tweet[i] = data.text[i]
                    tweets.time[i] = data.created_at[i]
             else:   
                if "full_text" in data.extended_tweet[i].keys():
                     tweets.tweet[i]=data.extended_tweet[i]["full_text"]
                     tweets.time[i] = data.created_at[i]
                    
                else:
                     tweets.tweet[i]=data.text[i] 
                     tweets.time[i] = data.created_at[i]
       else:
            if 'extended_tweet' in data.retweeted_status[i].keys():
                if "full_text" in data.retweeted_status[i]['extended_tweet'].keys():
                    tweets.tweet[i]= data.retweeted_status[i]['extended_tweet']["full_text"]
                    tweets.time[i] = data.created_at[i]
            else:
                 tweets.tweet[i] = data.retweeted_status[i]['text']
                 tweets.time[i] = data.created_at[i]
                    
       #print( tweets.tweet[i])
       #print('----------------\n')
                    

          

In [25]:
tweets = tweets.sort_values('time', ascending=False)

In [26]:
tweets=tweets.drop_duplicates()

In [27]:
sum(tweets.duplicated())

0

In [28]:
print('The time range tweets written is from', min(tweets.time),'to', max(tweets.time))

The time range tweets written is from 2019-04-17 14:39:20 to 2019-05-12 15:15:48


In [30]:
tweets.to_hdf('/Users/shahla/Dropbox/SharpestMinds/tweets.h5', key='df', mode='w')  

/Users/shahla/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:2378: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->['time', 'tweet']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [10]:
tweets = pd.read_hdf('/Users/shahla/Dropbox/SharpestMinds/tweets.h5')


In [11]:
tweets.shape

(68835, 2)

In [12]:
tweets.head()

,time,tweet
80018,2019-05-12 15:15:48,@MigunaMiguna @RailaOdinga @WilliamsRuto I rea...
80017,2019-05-12 15:15:48,"The grip of two old parties, the EPP &amp; the..."
80016,2019-05-12 15:15:48,North Carolina Blowback Intensifies on Richard...
80015,2019-05-12 15:15:48,@kae_stoddard To begin with it is a race-based...
80014,2019-05-12 15:15:48,Iran threatening American forces with ballisti...


In [13]:
tweets.iloc[100].time

Timestamp('2019-05-12 15:15:33')

In [14]:
tweets.time[101]

KeyError: 101

In [15]:
tweets.isnull().sum().sum()

0

# Text Pre-Processing
Depending on your downstream task, cleaning and pre-processing text can involve several different components.  Here are a few important components of Natural Language Processing (NLP) pipelines.

1.  Removing tags: unnecessary content like HTML tags

2.  Removing accented characters: other languages such as French, convert ASCII

3. Removing special characters: adds noise to text, use simple regular expressions (regexes)

4.  Stemming and lemmatization: Stemming remove prefixes and suffixes of word stems (i.e. root words), ex. WATCH is the root stem of WATCHES, WATCHING, and WATCHE.  Lemmatization similar but lexicographically correct word (present in the dictionary).

5.  Expanding contractions: helps text standardization, ex. do not to don’t and I would to I’d

6.  Removing stopwords: Words without meaningful significance (ex. a, an, the, and) but high frequency.

Additional pre-processing: tokenization, removing extra whitespaces,  lower casing and more advanced operations like spelling corrections, grammatical error corrections, removing repeated characters.

In [16]:
import re, string, unicodedata
import nltk
import contractions
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer


In [17]:

import string

import nltk
from nltk.corpus import stopwords
from nltk import re
#import regex, utils
#from profanity_filter import ProfanityFilter

#----------------------------------------------------------------------------

#----------------------------------------------------------------------------
class TwitterPreprocessor:

    def __init__(self, text: str):
        self.text = text
# =============================================================================
#         self.pf = self._init_profanity_filter()
# 
#     @staticmethod
#     def _init_profanity_filter():
#         pf = ProfanityFilter()
#         pf.censor_char = ' '
#         pf.censor_whole_words = True
#         return pf
# =============================================================================

    def fully_preprocess(self):
        return self \
            .remove_hyphens() \
            .remove_urls() \
            .remove_mentions() \
            .remove_hashtags() \
            .remove_twitter_reserved_words() \
            .remove_punctuation() \
            .remove_single_letter_words() \
            .remove_stopwords() \
            .remove_numbers() 
            
    def remove_hyphens(self):
        self.text=self.text.replace("-"," ")
        return self
    def remove_urls(self):
        self.text = re.sub(pattern=re.compile(
        r'(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))'
        r'[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9]\.[^\s]{2,})'), repl='', string=self.text)
        return self


    def remove_punctuation(self):
        self.text = self.text.translate(str.maketrans('', '', string.punctuation))
        return self
 
    def remove_mentions(self):
        self.text = re.sub(pattern=re.compile(r'@\w*'), repl='', string=self.text)
        return self

    def remove_hashtags(self):
        self.text = re.sub(pattern=re.compile(r'#*'),repl='',string=self.text)
        return self
#('RT @[\w_]+: ', '', tweet)
#re.sub(pattern=re.compile(r'RT @[\w_]+: '),repl=' ',string=self.text)
#z = lambda x: re.compile('\#').sub('', re.compile('RT @').sub('@', x, count=1).strip())
    def remove_twitter_reserved_words(self):
        self.text = re.sub('RT','',self.text,flags=re.IGNORECASE)
 #       self.text = re.sub(pattern=re.compile('RT @[\w_]+: '),repl=' ',string=self.text)
        return self

    def remove_single_letter_words(self):
        self.text = re.sub(pattern=re.compile(r'(?<![\w\-])\w(?![\w\-])'),repl=' ',string=self.text)
        return self

#     def remove_blank_spaces(self):
#         self.text = re.sub(pattern=re.compile(r'\s{2,}|\t'),repl='',string=self.text)
#         return self

    def remove_stopwords(self, extra_stopwords=None):
        if extra_stopwords is None:
            extra_stopwords = []
        text = nltk.word_tokenize(self.text)
        stop_words = set(stopwords.words('english'))

        new_sentence = []
        for w in text:
            if w not in stop_words and w not in extra_stopwords:
                new_sentence.append(w)
        self.text = ' '.join(new_sentence)
        return self

            
# =============================================================================
#     def remove_profane_words(self):
#         self.text = self.pf.censor(self.text)
#         self.remove_blank_spaces()
#         return self
# =============================================================================

    def remove_numbers(self, preserve_years=False):
        text_list = self.text.split(' ')
        for text in text_list:
            if text.isnumeric():
                if preserve_years:
                    if utils.is_year(text):
                        text_list.remove(text)
                else:
                    text_list.remove(text)

        self.text = ' '.join(text_list)
        return self


In [18]:
tweets[99:110]

,time,tweet
79910,2019-05-12 15:15:33,.@RahulGandhi''s modest and unpresuming compor...
79911,2019-05-12 15:15:33,Impeach @RepMaxineWaters The Crook!\n#MaxineWa...
79912,2019-05-12 15:15:33,Disappointing to see his Excellency hon @rasht...
79913,2019-05-12 15:15:33,@LarryGlickman Or one might explore the possib...
79914,2019-05-12 15:15:33,#ForcedBirth Fun Facts 2 con't\nBoth states (G...
79915,2019-05-12 15:15:33,Your house will belong to us\nYour job will be...
79899,2019-05-12 15:15:32,This isn’t speculation—it’s the opinion of Geo...
79898,2019-05-12 15:15:32,Earth is round. Round is football. My dadi liv...
79896,2019-05-12 15:15:32,Do you suppose Pence gets his own joke? https:...
79895,2019-05-12 15:15:32,"Come on, guys! Leave the BBC alone. Election r..."


In [19]:
tweets.iloc[101].tweet

'Disappointing to see his Excellency hon @rashtrapatibhvn voting. There have been precedents in the past but that does not make it right. The hon President must be above politics. If he is voting for A party (eg BJP) how will he call one over the other incase of a hung parliament'

In [20]:
text=tweets.tweet[101]
print(text)

KeyError: 101

In [22]:
#from twitter_preprocessor import TwitterPreprocessor

tweets.loc[:,'processed'] = tweets.tweet.apply(lambda x:TwitterPreprocessor(x).fully_preprocess().text)

#p.fully_preprocess()
#print(p.text)


In [25]:
from text_normalizer import normalize_corpus
tweets.loc[:,'normalized']=normalize_corpus(tweets.processed)

In [ ]:
tweets.to_hdf('/Users/shahla/Dropbox/SharpestMinds/tweets-processed.h5', key='df', mode='w') 


In [46]:
# from twitter_preprocessor import TwitterPreprocessor

# p = TwitterPreprocessor(tweets.tweet[12])

# p.fully_preprocess()
# print(p.text)


In [59]:
tweets.tweet[12]

'So, these are the peacekeepers in #Afghanistan on their mission. Very very disturbing‼️\U0001f92e The Kill Team Photos https://t.co/iPVMn1b7Rm via @RollingStone'

In [214]:
tweets.tweet[12]

'So, these are the peacekeepers in #Afghanistan on their mission. Very very disturbing‼️\U0001f92e The Kill Team Photos https://t.co/iPVMn1b7Rm via @RollingStone'

In [3]:
data = pd.read_hdf('/Users/shahla/Dropbox/SharpestMinds/tweets-processed.h5')

In [4]:
data = data.iloc[0:100]

In [5]:
import gensim #downloaded pretrained model


/Users/shahla/anaconda3/lib/python3.6/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [6]:
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary = True)

In [10]:
result = model.most_similar(positive=['woman', 'king'], negative=['man'])
print("{}: {:.4f}".format(*result[0]))

queen: 0.7118


In [ ]:
result = model.most_similar_cosmul(positive=['woman', 'king'], negative=['man'])
print("{}: {:.4f}".format(*result[0]))
#queen: 0.8965

In [ ]:
print(model.doesnt_match("breakfast cereal dinner lunch".split()))
# cereal

In [ ]:
similarity = model.similarity('woman', 'man')
#  similarity > 0.8
# True

In [ ]:
result = model.similar_by_word("cat")
print("{}: {:.4f}".format(*result[0]))
# dog: 0.8798
sentence_president = 'The president greets the press in Chicago'.lower().split()
similarity = model.wmdistance(sentence_obama, sentence_president)
print("{:.4f}".format(similarity))
# 3.4893
distance =model.distance("media", "media")
print("{:.1f}".format(distance))
# 0.0
sim = model.n_similarity(['sushi', 'shop'], ['japanese', 'restaurant'])
print("{:.4f}".format(sim))
# 0.7067
vector = model['computer']  # numpy vector of a word
vector.shape
# (100,)
vector = model.wv.word_vec('office', use_norm=True)
vector.shape
# (100,)

In [13]:
?model.wv.word_vec

In [7]:
tweets=[tweet.split() for tweet in data.normalized]

In [8]:

filtered_tweets = [[word for word in tweet if word in model.vocab] for tweet in tweets];

In [9]:
len(filtered_tweets)

100

In [ ]:
#https://stackoverflow.com/questions/32759712/how-to-find-the-closest-word-to-a-vector-using-word2vec
your_word_vector = array([-0.00449447, -0.00310097, 0.02421786, ...], dtype=float32)
model.most_similar(positive=[your_word_vector], topn=1)

In [16]:
model.similar_by_vector(word_vecs[2],topn=10, restrict_vocab=None)

[('kenya', 1.0),
 ('zimbabwe', 0.6655669808387756),
 ('uganda', 0.6655056476593018),
 ('naija', 0.6574218273162842),
 ('abuja', 0.6500369906425476),
 ('9ja', 0.6490509510040283),
 ('nigeria', 0.6328322887420654),
 ('ethiopia', 0.6305173635482788),
 ('zambia', 0.6301796436309814),
 ('africa', 0.6298450827598572)]

In [14]:
vectorized_titles = pd.DataFrame(columns=["Vectors"])
from sklearn import preprocessing
for tweet in filtered_tweets:
    word_vecs = [model[word] for word in tweet]
    title_vec = [np.zeros(300)]
    if len(word_vecs) == 0:
        title_vec = [np.zeros(300)]
    else: 
        title_vec = preprocessing.normalize(sum(word_vecs).reshape(1, -1))
    vectorized_titles = vectorized_titles.append({'Vectors': title_vec}, ignore_index=True)

In [ ]:

dictionary = dict(zip(post_titles, filtered_word_list))
vectorized_titles = pd.DataFrame(columns=["Titles", "Vectors"])
for title in post_titles: 
    word_vecs = [model[word] for word in dictionary[title]]
    if len(word_vecs) == 0:
        title_vec = [np.zeros(300)]
    else: 
        title_vec = normalize(sum(word_vecs).reshape(1, -1))
    vectorized_titles = vectorized_titles.append({'Titles': title, 'Vectors': title_vec}, ignore_index=True)


In [ ]:
vectorized_titles.to_pickle("/Users/angelateng/Dropbox/AMBER/SharpestMinds/vectorized_titles.pkl")